# IMPORT

In [20]:
import numpy as np
import pandas as pd
import math
import os
from datetime import datetime
from tqdm import tqdm
import pypyodbc


In [25]:
[x for x in pypyodbc.drivers() if x.startswith('Microsoft Access Driver')]

['Microsoft Access Driver (*.mdb, *.accdb)']

## mdb

In [26]:
mdb_path = '../../現有的Data/mdb/mdb/'
mdb_file_list = os.listdir(mdb_path)
print(mdb_file_list)

['d1.mdb', 'd2.mdb']


In [27]:
mdb_dir_path = 'D:\\NCKU\\3_1\\iir_lab\\112專題\\task1\\現有的Data\\mdb\\mdb\\' ## Microsoft Access 檔案名稱
temp_labdata_list = np.array([[0,0,0,0,0,0]])

for mdb_file_name in mdb_file_list:
    connection_string = r'DRIVER={Microsoft Access Driver (*.mdb)};DBQ=%s;' % (mdb_dir_path + mdb_file_name)
    conn = pypyodbc.win_connect_mdb(connection_string)
    SQL = 'select ChartNo, Specimen, SamplingTime, ItemCode, ItemName, TestValue from Labdata' 
    cur = conn.cursor()

    cur.execute(SQL)

    mdb_data = cur.fetchall()
    desc = cur.description
    cur.close()
    conn.close()

    df_column = pd.DataFrame(desc)
    df_column[0]

    df = pd.DataFrame(mdb_data, columns=df_column[0])
    df = df.to_numpy()
    print(df[10000])
    temp_labdata_list = np.append(temp_labdata_list,df, axis = 0)
#     df.head()

['06143629' '血液' '2011-03-14 07:44:14.907' '8202TCO2' 'TCO2' '31.1']
['06315235' '血液' '2016-07-02 01:28:10.257' '8202eGFR' 'eGFR' '≧90']


In [28]:
temp_labdata_list = np.delete(temp_labdata_list, 0, axis = 0)
print(temp_labdata_list[0:10])

[['05793660' '血液' '2011-06-03 16:56:50.023' '8210Baso' 'Baso' '0']
 ['05793660' '血液' '2011-06-03 16:56:50.023' '8210Blast' 'Blast' '0']
 ['05793660' '血液' '2011-06-03 16:56:50.023' '8210Eos' 'Eos' '1']
 ['05793660' '血液' '2011-06-03 16:56:50.023' '8210Lymph' 'Lymph' '49']
 ['05793660' '血液' '2011-06-03 16:56:50.023' '8210Meta' 'Meta' '0']
 ['05793660' '血液' '2011-06-03 16:56:50.023' '8210Mono' 'Mono' '7']
 ['05793660' '血液' '2011-06-03 16:56:50.023' '8210Myelo' 'Myelo' '0']
 ['05793660' '血液' '2011-06-03 16:56:50.023' '8210NRBC' 'NRBC' '0']
 ['05793660' '血液' '2011-06-03 16:56:50.023' '8210Pro' 'Pro' '0']
 ['05793660' '血液' '2011-06-03 16:56:50.023' '8210Seg' 'Seg' '43']]


In [29]:
for i in tqdm(range(0, temp_labdata_list.shape[0])):
    if str.isdigit(temp_labdata_list[i][0]):
        temp_labdata_list[i][0] = int(temp_labdata_list[i][0])
    else:
        print(temp_labdata_list[i][0])

100%|██████████████████████████████████████████████████████████████████| 16289285/16289285 [00:19<00:00, 828930.44it/s]


## extract

In [30]:
itemname_set = set(temp_labdata_list[:,4])
# print(itemname_set)
# print(len(itemname_set))

extract_path = '../raw_data/extracted_labdata'
column_names = ["patient_code", "Specimen", "SamplingTime", "ItemCode", "ItemName", "TestValue"]
special_list = []
for itemname in tqdm(itemname_set):
    file_itemname = str('')
    file_name_list = itemname.split('/')
    for i in file_name_list:
        print(i)
        file_itemname = file_itemname + i
        
    temp_ary = temp_labdata_list[np.where(temp_labdata_list[:,4] == itemname)]
    specimen_set = set(temp_ary[:,1])
    
    for specimen in specimen_set:
        target_ary = temp_ary[np.where(temp_ary[:,1] == specimen)]
        target_df = pd.DataFrame(target_ary, columns = column_names)
        # print(target_df)
#         break
        
        if os.path.isfile(extract_path + '/' + file_itemname + specimen + '.csv') == False:
            target_df.to_csv(extract_path + '/' + file_itemname + specimen + '.csv', index = False, encoding = 'utf_8_sig')
        else:
            special_list.append([file_itemname, itemname])
            target_df.to_csv(extract_path + '/' + file_itemname + specimen + '2.csv', index = False, encoding = 'utf_8_sig')
#     break

print(special_list)

  0%|                                                                                          | 0/376 [00:00<?, ?it/s]

HDL-C


  0%|▏                                                                                 | 1/376 [00:00<04:58,  1.26it/s]

BUN


  1%|▋                                                                                 | 3/376 [00:03<06:07,  1.02it/s]

...

AAT
AMON


 99%|███████████████████████████████████████████████████████████████████████████████▌| 374/376 [03:43<00:00,  2.42it/s]

GTT(75g)-2hr
Free kappa


100%|████████████████████████████████████████████████████████████████████████████████| 376/376 [03:44<00:00,  1.67it/s]

THb
[['CREA', 'CREA']]


In [15]:
print(type(special_list[0][0]))
print(type(special_list[0][1]))
print(os.path.exists(extract_path + '/' + 'Pro'))

<class 'str'>
<class 'str'>
False


# 下面還沒改好

## patient list

In [ ]:
patient_frame = pd.read_csv('../processing_data/label1/6/patient_list.csv', encoding = 'utf-8', engine = 'python')
big_patient_list = patient_frame['patient_code'].tolist()
big_patient_list = list(map(int, big_patient_list))

In [ ]:
print(len(big_patient_list))

## delete unuseful mdb records

In [ ]:
big_labdata_list = np.array([[0,0,0,0]])
for patient in tqdm(big_patient_list):
    temp_list = temp_labdata_list[np.where(temp_labdata_list[:,0] == patient)]
    big_labdata_list = np.append(big_labdata_list, temp_list, axis = 0)
big_labdata_list = np.delete(big_labdata_list, 0, axis = 0)

## make the set of itemname

In [ ]:
itemname_set = set(big_labdata_list[:,2])
print(itemname_set)
print(len(itemname_set))

#check
p_set = set(big_labdata_list[:,0])
#print(itemname_set)
print(len(p_set))

# extracting & caculate

In [ ]:
extract_path = '../processing_data/extracted_labdata_for6h'
column_names = ["patient_code", "SamplingTime", "ItemName", "TestValue"]
itemnum_sta = []
special_list = []
for itemname in itemname_set:
    if '/' in itemname:
        file_itemname = itemname.replace('/', '')
        special_list.append(itemname)
    else:
        file_itemmname = itemname
    temp_ary = big_labdata_list[np.where(big_labdata_list[:,2] == itemname)]
    temp_df = pd.DataFrame(temp_ary, columns = column_names)
    if os.path.exists(extract_path + '/' + file_itemmname) == False:
        os.makedirs(extract_path + '/' + file_itemmname)
    temp_df.to_csv(extract_path + '/' + file_itemmname + '/' + file_itemmname + '.csv', index = False, encoding = 'utf-8')
    itemnum_sta.append([itemname, temp_ary.shape[0]])
itemnum_sta = pd.DataFrame(itemnum_sta, columns = ["itemname", "total_num"])
itemnum_sta.to_csv('../calculation.csv', index = False, encoding = 'utf-8')
print(special_list)

# EXTRCT ALL ITEM INTO SEPERATE FILE

In [ ]:
#全部的labdata
big_labdata_list = np.array([[0,0,0,0]])
for filename in filelist:
    df = pd.read_csv(csv_path + filename, usecols = ['ChartNo', 'SamplingTime', 'ItemName', 'TestValue'])
    df = df.to_numpy()
    big_labdata_list = np.append(big_labdata_list,df, axis = 0)
big_labdata_list = np.delete(big_labdata_list, 0, axis = 0)

In [ ]:
print(big_labdata_list)
print(big_labdata_list.shape[0])

In [ ]:
print(big_labdata_list[1000000][1])

In [ ]:
#可以被使用的病人的labdata
big_labdata_list = np.array([[0,0]])
for filename in filelist:
    df = pd.read_csv(csv_path + filename, usecols = ['ChartNo', 'ItemName'])
    df = df.to_numpy()
    new_df = df[np.where(np.isin(df[:,0], big_patient_list))]
    big_labdata_list = np.append(big_labdata_list,new_df, axis = 0)
big_labdata_list = np.delete(big_labdata_list, 0, axis = 0)

In [ ]:
print(big_labdata_list)
print(big_labdata_list.shape[0])

In [ ]:
labdata_frame = pd.DataFrame(big_labdata_list[: , 0], columns = ['patient_code'])
labdata_frame = pd.concat([labdata_frame, pd.DataFrame(big_labdata_list[:, 1], columns = ['itemname'])], axis = 1)
labdata_frame.to_csv('../processing_data/mdb/labdata_to_csv.csv', index = False, encoding = 'utf-8')

In [ ]:
labdata_frame = pd.read_csv('../processing_data/mdb/labdata_to_csv.csv', encoding = 'utf-8', engine = 'python')
labdata_frame = labdata_frame.to_numpy()
print(labdata_frame)
print(labdata_frame.shape[0])

In [ ]:
#labdata檢測種類
itemname_set = set(labdata_frame[:,1])
print(itemname_set)
print(len(itemname_set))

In [ ]:
patient_count_list = []
itemname_list = list(itemname_set)
for i in range(0, len(itemname_set)):
    new_df = labdata_frame[np.where(labdata_frame[:,1] == itemname_list[i])]
    num = len(set(new_df[:,0]))
    print(num)
    patient_count_list.append(num)

In [ ]:
print(itemname_list)

In [ ]:
labdata_countlist = np.array([[0,0]])
for i in range(0, len(itemname_list)):
    labdata_countlist = np.append(labdata_countlist, [[patient_count_list[i], itemname_list[i]]], axis = 0)
labdata_countlist = np.delete(labdata_countlist, 0, axis = 0)
print(labdata_countlist)

In [ ]:
result_frame = pd.DataFrame(labdata_countlist[: , 1], columns = ['itemname'])
result_frame = pd.concat([result_frame, pd.DataFrame(labdata_countlist[: , 0], columns = ['patientcount'])], axis = 1)
result_frame.to_csv('../processing_data/mdb/mdb_count_result.csv', index = False, encoding = 'utf_8_sig')

#分檔

In [ ]:
csv_path = '../labdata_to_csv/labdata/'
filelist = os.listdir(csv_path)

In [ ]:
print(filelist)